## To do
- Fix image ID extraction.
  - Currently, img-ids are read from a directory listing.
  - Rather, use the img-ids from the CSV's dataframe - it's more fool-proof and avoids spurious subset errors.

- Run DenseNets and EfficientNets.
  - Ensure that you are saving the models at regular intervals - Colab disconnects randomly!

In [ ]:
import os
import tensorflow as tf
import keras.utils
import numpy as np
import cv2
import pandas as pd

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.test.is_built_with_cuda()
print(tf.version.VERSION)
import sys
sys.version

Num GPUs Available:  0
2.12.0


'3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]'

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import tensorflow as tf
tf.config.list_physical_devices('GPU')
# tf.debugging.set_log_device_placement(False)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10597184215341326724
xla_global_id: -1
]


[]

#Load Data


In [ ]:
base_path = os.getcwd()

## densenet 169

In [ ]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input
from keras import backend as K

from tensorflow.keras.applications.densenet import DenseNet169 # DenseNet121
from keras.callbacks import CSVLogger


IMG_SIZE=(224,224)

#model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
model = DenseNet169(
        include_top=False,
        #weights='imagenet',
        input_shape=(*IMG_SIZE, 3)
       )
#model.summary(line_length=150)

flatten = Flatten(name='flatten_step_1')
new_layer2 = Dense(2125, activation='softmax', name='my_dense_2')


inp2 = model.input
out2 = new_layer2(flatten(model.output)) #removed the new layer for ensemble

opt = keras.optimizers.Adam(learning_rate=1e-08)

model2 = Model(inp2, out2)

#model2.summary()
model2.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['acc']
)


# weight_save = keras.callbacks.ModelCheckpoint(base_path+'/Data/denseNet169NewLayersWeights/weights-epoch-{epoch:003d}.h5', save_weights_only=True, period=1)
# on_epoch_end_call = keras.callbacks.LambdaCallback(on_epoch_end=data_reader.on_epoch_end())
# csv_logger = CSVLogger('trainingDenseNet1691NewLayers.csv', append=True)


#model2.load_weights(base_path+'/Data/denseNet121Weights/weights-epoch-0085.h5')

# Removing dense layer
# Freezing all the other layers
# Adding 2 dense layers

modelWithoutTop = Model(model2.input, model2.get_layer('flatten_step_1').output)

for layer in modelWithoutTop.layers:
    layer.trainable=False
    
modelWithoutTop.trainable=False
#model_without_top.summary()

fc1 = Dense(4096,activation='relu', name='my_dense_6')(modelWithoutTop.output)
fc2 = Dense(2125,activation='softmax', name='my_dense_69')(fc1)

#one model file with 8192->1572
model2 = Model(inputs=modelWithoutTop.input, outputs=fc2)

#model2.summary()

model2.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['acc']
)


modelWithoutTop = Model(model2.input, model2.get_layer('my_dense_6').output)

modelWithoutTop.summary()

modelWithoutTop.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['acc']
)
modelWithoutT.load_weights('/content/drive/MyDrive/ImageCLEFmedical-23/weights/densenet169/weights-epoch-new-0022.h5')
#set train paramenters to false
# add dense 8192 
# add dense 200x
#modelWithoutTop.save_weights(base_path+'/Data/EnsembleWeights/DatadenseNet121-weights-epoch-0085-withoutTop.h5')
#extract features


Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_8 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_5[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_8[0][0]']       
                                )                                                          

In [ ]:
import csv
import time

class DataGenImageLoader(keras.utils.Sequence):
    def __init__(self, ids, encoder=None, mlb=None, batch_size=8, image_size=224):
        self.ids = ids
        self.batch_size = batch_size
        self.image_size = image_size
        
        if encoder is not None:
          self.encoder = encoder
        if mlb is not None:
          self.mlb = mlb

        self.on_epoch_end()
        
    def __load__(self, id_name):
        ## Path
        image_path = base_path + '/Captioning-Dataset_CLEF-2023/test'+'/'+id_name     
        #print(image_path)
        
        ## Reading Image        
        image = cv2.imread(image_path, 1)        
        
        if(image is None):
          print("Error: ",id_name)
        try:
          image = cv2.resize(image, (self.image_size, self.image_size)).astype('float32')
          #print(image.shape)
        except:
          print("Skipping image: ")
        # Convert to float
        image /= 255
        return image
    
    def __getitem__(self, index):#For returning batch of images and thier cui ids

        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        #print(len(files_batch))
        
        images = []
        ids = []
        for i,id_name in enumerate(files_batch):
            _img = self.__load__(id_name)
            # print(id_name)
            images.append(_img)   
            ids.append(id_name)
            
        images = np.array(images)

        # print("Batch Encoded CUI IDs", encoded_batch_cuis)
        return images, ids
    
    def on_epoch_end(self):
        pass
        #print("Reordering image ids")
        #np.random.shuffle(self.ids)
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))

op_file = open(base_path+'test_densenet121_featureVectors1.csv', mode='w')
op_writer = csv.writer(op_file, delimiter=',')

img_id = [f'ImageCLEFmedical_Caption_2023_test_{str(i).zfill(6)}.jpg' for i in range(1, 10474)]
dataLoader = DataGenImageLoader(img_id, batch_size=3)



startTime = time.time()
totalTime = 0
for batchNum in range(0,3491):
    if(batchNum%500==0 and batchNum != 0):
        op_file.close()
        op_file = open(base_path+'test_densenet121_featureVectors1.csv', mode='a')
        op_writer = csv.writer(op_file, delimiter=',')
        
    start = time.time()
    print("Current batch: ", batchNum)
    predImgs, ids = dataLoader.__getitem__(batchNum)


    predictions = modelWithoutTop.predict(predImgs)
    print(len(predictions[0]))


    for i in range(len(ids)):
        temp = [ids[i]]
        temp = np.array(temp)
        temp = np.append(temp, predictions[i])
        op_writer.writerow(temp)
    
    totalTime += time.time()-start
    meanEpochTime = totalTime / (batchNum+1)
    timeLeft = (meanEpochTime*(3491 - batchNum + 1)) 
    print("batch ", batchNum, "took: ", time.time()-start,"\nETA: ", timeLeft)

op_file.close()

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 1s 796ms/step
4096
batch  1602 took:  2.0784847736358643 
ETA:  3931.9065312527155
Current batch:  1603
1/1 [==============================] - 1s 797ms/step
4096
batch  1603 took:  1.6396944522857666 
ETA:  3929.30716178557
Current batch:  1604
1/1 [==============================] - 1s 751ms/step
4096
batch  1604 took:  2.2342848777770996 
ETA:  3927.408423098745
Current batch:  1605
1/1 [==============================] - 1s 752ms/step
4096
batch  1605 took:  2.4272685050964355 
ETA:  3925.736017713511
Current batch:  1606
1/1 [==============================] - 1s 829ms/step
4096
batch  1606 took:  2.1182966232299805 
ETA:  3923.700053945267
Current batch:  1607
1/1 [==============================] - 1s 1s/step
4096
batch  1607 took:  2.130357265472412 
ETA:  3921.678123526757
Current batch:  1608
1/1 [==============================] - 1s 766ms/step
4096
batch  1608 took:  1.6711089611053467 
ETA: